In [40]:
# Распред обручей
import pandas as pd
import numpy as np
import random

shops = pd.read_excel('C:\анаконда\shop_list.xlsx', dtype='int')
                     
# достаем Тпг+размер c количеством которые нужно считать из накладной 
tmp_df = pd.read_excel('C:/рушники/123.xlsx', dtype='str')
tmp_df['tpg'] = tmp_df['Товарная подгруппа'].str.cat(tmp_df['Размер'], sep="_")

#del tmp_df['№ Поставщика']
tmp_df['tpg'] = tmp_df['tpg'].str.replace('.', ',')# меняем точки на запятые
tmp_df.sort_values(by=['tpg'], inplace=True)
tpg = tmp_df['tpg'].unique().tolist()
tpg.sort()
tpg_dic = tmp_df['tpg'].value_counts().to_dict()

# Загружаем нормы и остатки по нужным тпг чтобы посчитать дефицит
n_link =  r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\нужное\матрица обручей.xlsx'
norms = pd.read_excel(n_link, sheet_name='гладкая_плоская размеры_тпг', skiprows=3, usecols=['номер', *tpg]) 
stock = pd.read_excel(r'C:\Остатки\Обручальные кольца.xlsx', usecols=['КодСклада','ТоварнаяПодгруппа','РазмерИзделия'], dtype='str')
stock.columns = ['КодСклада', 'Товарная подгруппа', 'Размер']
stock['Размер'] = stock['Размер'].str.replace('.', ',')
stock['tpg'] = stock['Товарная подгруппа'].str.cat(stock['Размер'], sep="_")
stock.drop(columns=['Товарная подгруппа','Размер'], inplace=True)
stock = stock[stock['tpg'].isin(tpg)]
stock = stock.groupby(by=['КодСклада', 'tpg'])['tpg'].aggregate('count').unstack(fill_value=0.0)
stock.reset_index(inplace=True)
stock = stock.astype('int')
stock.columns.values[0] = 'номер'
stock = stock[norms.columns.tolist()]
norms = shops.merge(norms, on='номер', how='left')
stock = shops.merge(stock, on='номер', how='left').fillna(0)
norms.set_index('номер', inplace=True)
stock.set_index('номер', inplace=True)

norms = norms.applymap(lambda x: x + 1 if x == 0 else round(x * 1.3))# для распреда того что не входит

demand = norms - stock
demand = demand.applymap(lambda num: 0 if num < 0 else num)

def raspred_dict(df):
    emp = []
    cols = df.columns.tolist()
    for col in cols:
        emp.append(list(np.repeat(df.index.to_numpy(),df[col].values.astype(int))))
    return dict(zip(cols, emp))

no7171_dict = raspred_dict(demand)

def dictu(x):    
    if tpg_dic[x] > len(no7171_dict[x]):
        over = tpg_dic[x] - len(no7171_dict[x])
        no7171_dict[x] = [*no7171_dict[x], *([7171] * over)]        
    elif tpg_dic[x] < len(no7171_dict[x]):
        no7171_dict[x] = no7171_dict[x][:tpg_dic[x]]        
    return no7171_dict[x]

lst = [dictu(x) for x in tpg]

tmp_df['Период реализации'] = np.array([item for sublist in lst for item in sublist])
name = 'C:/рушники/' + str(random.randrange(5000,10000)) + '.xlsx'
tmp_df.to_excel(name, sheet_name='Движение товара', index=False)

In [43]:
# join files in folder
import os
import pandas as pd

path = 'C:/рушники/соединить'
  
files = [path + '/' + x for x in os.listdir(path)]
all_files = pd.concat([pd.read_excel(x) for x in files])
all_files.to_excel(path + '/all.xlsx', sheet_name='Движение товара', index=False)

In [8]:
# Распред букв или зодиаков по одному направлению
import pandas as pd
import numpy as np
import random

# get stockdata, goods receipt, shops list, stock capability
prih = pd.read_excel('C:/рушники/ИФ ЗОДИАК.xlsx')
prih.sort_values(by=['Товар'], inplace=True)
prih['Описание 3'] = prih['Описание 3'].str.lower()

letters = prih['Описание 3'].unique().tolist()
tovs = prih['Товар'].unique().tolist()
tovs.sort()

stock = pd.read_excel('C:/Остатки/Буквы и Зодиаки.xlsx')
stock = stock[stock['ТоварНо'].isin(tovs)]
stock.rename(columns={'Описание3': 'Описание 3'}, inplace=True)
stock['Описание 3'] = stock['Описание 3'].str.lower()

#create automatic demand calculation here
#norms = pd.read_excel('C:/Остатки/norm_bz.xlsx', sheet_name='norm_z', index_col= 'Название из NAV', usecols=['Название из NAV', 'sum_bk'])
#norm_dict = norms.sum_bk.apply(lambda x: round(x/12)).to_dict()

#estimate sheetname and filename for demand
tn = prih.loc[1, 'Товарное направление'].lower()
fname = 'C:/Остатки/' + prih.loc[1, 'Дизайн'][3:].lower() + '.xlsx'

demand = pd.read_excel(fname, sheet_name=tn, skiprows=1, usecols=['Наполняха', 'Код', *letters], index_col='Код')


#Create two dictionaries tov dict and letter dict
stock_dict = dict(zip(tovs,[stock[stock['ТоварНо'] == tov]['КодСклада'].tolist() for tov in tovs]))
df = prih.groupby(by=[ 'Товар', 'Описание 3'])['Товар'].count().rename('count')
df = df.reset_index()

tov_dict = dict(zip(df.Товар.tolist(), df[['Описание 3', 'count']].values.tolist())) #(tov:[letter, demand])

#letter: dataframe with columns:store number, filling rate, demand(positive num is overstock, negative num is demand)
letter_dict = dict(zip(letters,[demand[['Наполняха', letter]] for letter in letters]))

def get_shoplist(tov):
    """Get sorted by letter demand and stock list of stores with zero stock of tov.
    Changes letter dict according to this list. Two dictionaries needed to perform func"""
    
    letter = tov_dict[tov][0]
    lenth = tov_dict[tov][1]
    shoplist_full = letter_dict[letter].sort_values(by=[letter, 'Наполняха']).index.tolist()
    shoplist = [i for i in shoplist_full if i not in stock_dict[tov]]
    shop_len = len(shoplist)
    
    if shop_len > lenth:
        shoplist = shoplist[:lenth]
    elif len(shoplist) < lenth:
        over = lenth - shop_len
        shoplist = [*shoplist,*([7171] * over)]
    for shop in shoplist:
        if shop != 7171:
            letter_dict[letter][letter][shop] += 1
    return shoplist
      
lst = [get_shoplist(x) for x in tovs]
prih['Период реализации'] = np.array([item for sublist in lst for item in sublist])

name = 'C:/рушники/' + str(random.randrange(5000,10000)) + '.xlsx'
prih.to_excel(name, sheet_name='Движение товара', index=False)
    

C:\Users\Dotsenko.Semen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [42]:
# Распред по товам по две три штуки в шоплист вторым столбцом указать количество столбец "кол"
import pandas as pd
import numpy as np
import random

# create distribution quantities dictionary
invoice = pd.read_excel('C:/рушники/СЕРЬГИ.xlsx')
invoice.sort_values(by=['Товар'], inplace=True)
arts_dic = invoice['Товар'].value_counts().to_dict()

# create norms dataframe
shops = pd.read_excel('C:\анаконда\shop_list_doubled.xlsx', dtype='int')
shops.sort_values(by='кол',ascending=False, inplace=True)

num_dict = shops.кол.value_counts().sort_index(ascending=False).to_dict()
cols = list(invoice['Товар'].unique())

norms = pd.concat(
[pd.DataFrame(
    data=np.full((num_dict[i], len(cols)), i), 
    columns=cols)
 for i in list(num_dict)]
)  

norms.index = shops['номер']

# create stock df identical in shape to norms df
stock = pd.read_excel(r'C:\Остатки\Товы.xlsx', usecols=['КодСклада','ТоварНо'])
stock = stock[stock['КодСклада'].isin(norms.index.tolist()) & 
              stock['ТоварНо'].isin(cols)]

pivot = pd.pivot_table(stock,index='КодСклада', columns='ТоварНо', aggfunc=len, fill_value=0).reset_index()

stock = pd.DataFrame(data=shops['номер'].values, columns=['КодСклада']).merge(pivot, how='left')
stock.set_index('КодСклада',inplace=True)
stock.fillna(0, inplace=True)

if cols != stock.columns.to_list():
    add_cols = [x for x in cols if x not in stock.columns.to_list()]
    add_df = pd.DataFrame(data=np.zeros((stock.shape[0], len(add_cols))), columns=add_cols, index=stock.index)
    stock = stock.join(add_df)
    
stock = stock[cols]

# demand 
demand = norms - stock

# distribution
def repeat_series(series):
    return np.repeat(series.index, series.values).tolist()

empty = []
for col in demand.columns.tolist():
    shop_lst = repeat_series(demand[col].loc[lambda x: x > 0])
    diff = arts_dic[col] - len(shop_lst)
    if diff > 0:
        shop_lst.extend(list(np.full(diff, 7171)))
    else:
        shop_lst = shop_lst[:arts_dic[col]]
    empty.extend(shop_lst)

invoice['Период реализации'] = empty

# save and exit
name = 'C:/рушники/' + str(random.randrange(5000,10000)) + '.xlsx'
invoice.to_excel(name, sheet_name='Движение товара', index=False)

In [142]:
#Делим вырицу
path = 'C:/рушники/вырица.xlsx'
df = pd.read_excel(path, index_col=0)
df.fillna(0, inplace=True, downcast='infer') #либо менять dtype
df.sort_index(inplace=True)
cols = df.columns.tolist()

empty = []
for x in df.index:
    lst = np.repeat(cols, df.loc[x].values)
    empty = np.append(empty, lst)
    
path = 'C:/рушники/123.xlsx'
df1 = pd.read_excel(path)
df1.sort_values(by='Артикул', inplace=True)
df1['Период реализации'] = empty

# save and exit
name = 'C:/рушники/' + str(random.randrange(5000,10000)) + '.xlsx'
df1.to_excel(name, sheet_name='Движение товара', index=False)

In [36]:
# РАСПРЕД ПО МАГАМ ТОПАМ
import pandas as pd
import numpy as np

n_of_shops = 50

invoice = 'C:/рушники/123.xlsx'
fpath = 'C:/Остатки/наполняемость с 10012020.xlsx'

# накладная сортировка, сводная ТГ-тов-кол-во для распреда
prih = pd.read_excel(invoice)
prih['Тип изделия 1'] = prih['Тип изделия 1'].apply(lambda x: 'СЕРЬГИ' if x in ['ПИРСИНГ', 'СЕРЬГИ-КОНГО'] else x)
prih['Описание 3'] = prih['Товарное направление'] + '_' + prih['Тип изделия 1']
prih.sort_values(by=['Описание 3', 'Товар'], inplace=True)

prih_summary = pd.DataFrame(prih.groupby(by=['Описание 3', 'Товар'])['Товар'].count().rename('quantity')).reset_index()
prih_summary.sort_values(by=['Описание 3', 'Товар'], inplace=True)

# {ТГ: [магазины с большими нормами по тг, разрешенные к распреду, наполняемость<100%, сортировка по наполняемости]}
fpath = 'C:/Остатки/наполняемость с 10012020.xlsx'

lst = pd.read_excel(fpath, skiprows=2,nrows=0).columns.tolist()
lst = [str(x) for x in lst]#защита от ошибки если в ячейке цифра
point_column = max(i for i, v in enumerate(lst) if 'Кольцо' in v)

cols = [1, 2] + list(range(4, 10)) + list(range(point_column, point_column + 6))
#создаем список тг
type1 = ['КОЛЬЦО',
'КОЛЬЦО ОБРУЧАЛЬНОЕ',
'КОЛЬЦО ПЕЧАТКА',
'ПОДВЕС ДЕКОРАТИВНЫЙ',
'ПОДВЕС КУЛЬТОВЫЙ',
'СЕРЬГИ']

tg_list = []

for tn in ['ИФ', 'БК']:
    for t in type1:
        tg_list.append(tn + '_' + t)

def read_norms(tn):
    """Создает датафрейм по ТН с наполняемостью и нормами по ТГ, франчайзинг, закрытые и мусульманские удаляются"""
    names = [i for i in tg_list if tn in i]
    df = pd.read_excel(fpath, sheet_name=tn, skiprows=3, usecols=cols, index_col='Код')
    df = df[~df.блок.isin(['ф', 'м', 'з', 'д', 'м/з'])].drop(columns='блок')
    arrays = [['Нормы','Наполняемость'], names]
    df.columns = pd.MultiIndex.from_product(arrays)
    return df

IF, BK = read_norms('ИФ'), read_norms('БК')

"""Два варианта отбора магов: 1 - сначала выбрать количество по нормам, а потом убрать с наполняемостью меньше 100%; 
2 - убрать маги меньше 100% и выбрать количество. В 1 случае список магов к каждому ТГ будет разной длины и нужно делать 
словарь; во 2 случае списки к каждому тг одинаковой длины и можно сделать датафрейм. Реализуется 2 случай -  
датафрейм из словаря"""

df = IF.join(BK, how='outer')
shop_lsts = {}
keys = prih_summary['Описание 3'].unique()
for x in keys:
    filt = df.xs(x, level=1, axis=1).sort_values(by='Нормы').head(50)
    shop_lsts[x] = filt.index[filt['Наполняемость'] < 1.05].to_list()#наполняху менять тут

# Остатки реальные + накладные
cols=["КодСклада", "НазваниеСклада", "ТоварНо","СерийныйНомер","Проект"]
stock = pd.read_csv(r'C:\анаконда\tmp.csv', delimiter=';', encoding='utf-8',low_memory=False, usecols=cols)
stock = stock[stock.Проект.isin(['ЗОЛОТОЙ','ОРИОН','ЗОЛОТО','ТАЛАНТ', 'НОУНЕЙМ'])].sort_values('КодСклада')
del stock['Проект']

stock.dropna(inplace=True)
stock.КодСклада = stock.КодСклада.astype('int64')

stock_centr = stock[stock.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
stock = stock[~stock.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
stock = stock[["КодСклада", "ТоварНо", "СерийныйНомер"]]
del stock_centr['КодСклада']

cols=["Серийный Но.","Склад-приемник","Номер ЗЛ"]
zab = pd.read_excel(r'C:\анаконда\zab.xlsx', skiprows=4, usecols=cols).sort_values(by="Номер ЗЛ",ascending = False)
zab.rename(columns={"Серийный Но.":'СерийныйНомер', "Склад-приемник":'КодСклада',"Номер ЗЛ":'ЗЛ'}, inplace=True) 
del zab['ЗЛ']

stock_centr = pd.merge(stock_centr,zab,on=['СерийныйНомер'],how = 'inner')#inner join оставит только неперемещенный товар
stock_centr = stock_centr[["КодСклада",  "ТоварНо","СерийныйНомер"]]
stock = pd.concat([stock,stock_centr])
#оставим только маги-топы и товы из накладной
unique_lst = list(set([val for vals in shop_lsts.values() for val in vals]))
stock = stock[stock.КодСклада.isin(unique_lst)].drop(columns='СерийныйНомер')
stock = stock[stock.ТоварНо.isin(prih_summary.Товар.tolist())]
#формат колонок кодсклада и товы
stock = pd.pivot_table(stock,index='КодСклада', columns='ТоварНо', aggfunc=len, fill_value=0).reset_index()
stock = stock.rename_axis(None, axis=1).set_index('КодСклада')
if set(prih_summary.Товар.values) != set(stock.columns):
    cols = set(prih_summary.Товар.values) - set(stock.columns)
    stock = stock.assign(**dict(zip(list(cols), np.zeros(len(cols), dtype='int64'))))

# Распределение
def distribution(tg):
    """распред по списку магов ТГ"""
    tov_dict = dict(prih_summary[prih_summary['Описание 3'] == tg].iloc[:,[1, 2]].values)
    cols = list(tov_dict.keys())    
    shops = shop_lsts[tg]
    tn_stock = stock.loc[shops, cols]
    emp_lst = []
    for col in tn_stock.columns.tolist():
        shop_lst = tn_stock[col].loc[lambda x: x == 0].index.tolist()
        lenth = len(shop_lst)
        diff = tov_dict[col] - lenth
        
        if diff > 0:
            shop_lst.extend(list(np.full(diff, 7171)))
        else:
            shop_lst = shop_lst[:tov_dict[col]]
            
        emp_lst.extend(shop_lst)
    return emp_lst
    
distr_lst = []
for x in list(shop_lsts.keys()):
    distr_lst.extend(distribution(x))

prih['Период реализации'] = distr_lst

# save and exit
name = 'C:/рушники/' + str(np.random.randint(5000, 10000)) + '.xlsx'
prih.to_excel(name, sheet_name='Движение товара', index=False)

KeyError: 'Passing list-likes to .loc or [] with any missing labels is no longer supported, see https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike'

In [44]:
# РАСПРЕД ПО всем магам
import pandas as pd
import numpy as np

invoice = 'C:/рушники/123.xlsx'
fpath = 'C:/Остатки/наполняемость с 10012020.xlsx'

# накладная сортировка, сводная ТГ-тов-кол-во для распреда
prih = pd.read_excel(invoice)
prih['Тип изделия 1'] = prih['Тип изделия 1'].apply(lambda x: 'СЕРЬГИ' if x in ['ПИРСИНГ', 'СЕРЬГИ-КОНГО'] else x)
prih['Описание 3'] = prih['Товарное направление'] + '_' + prih['Тип изделия 1']
prih.sort_values(by=['Описание 3', 'Товар'], inplace=True)

prih_summary = pd.DataFrame(prih.groupby(by=['Описание 3', 'Товар'])['Товар'].count().rename('quantity')).reset_index()
prih_summary.sort_values(by=['Описание 3', 'Товар'], inplace=True)

# {ТГ: [магазины, разрешенные к распреду, сортировка по наполняемости]}
fpath = 'C:/Остатки/наполняемость с 10012020.xlsx'

lst = pd.read_excel(fpath, skiprows=2,nrows=0).columns.tolist()
lst = [str(x) for x in lst]#защита от ошибки если в ячейке цифра
point_column = max(i for i, v in enumerate(lst) if 'Кольцо' in v)

cols = [1, 2] + list(range(point_column, point_column + 12))
#создаем список тг
type1 = ['КОЛЬЦО',
'КОЛЬЦО ОБРУЧАЛЬНОЕ',
'КОЛЬЦО ПЕЧАТКА',
'ПОДВЕС ДЕКОРАТИВНЫЙ',
'ПОДВЕС КУЛЬТОВЫЙ',
'СЕРЬГИ']

tg_list = []

for tn in ['ИФ', 'БК']:
    for t in type1:
        tg_list.append(tn + '_' + t)

def read_norms(tn):
    """Создает датафрейм по ТН с наполняемостью и разрешениями по ТГ, франчайзинг, закрытые и мусульманские удаляются"""
    names = [i for i in tg_list if tn in i]
    df = pd.read_excel(fpath, sheet_name=tn, skiprows=3, usecols=cols, index_col='Код')
    df = df[~df.блок.isin(['ф', 'м', 'з', 'д', 'м/з'])].drop(columns='блок')
    arrays = [['Нормы','Наполняемость'], names]
    df.columns = pd.MultiIndex.from_product(arrays)
    return df

IF, BK = read_norms('ИФ'), read_norms('БК')

"""Два варианта отбора магов: 1 - сначала выбрать количество по нормам, а потом убрать с наполняемостью меньше 100%; 
2 - убрать маги меньше 100% и выбрать количество. В 1 случае список магов к каждому ТГ будет разной длины и нужно делать 
словарь; во 2 случае списки к каждому тг одинаковой длины и можно сделать датафрейм. Реализуется 1 случай -  
датафрейм из словаря"""

df = IF.join(BK, how='outer')
shop_lsts = {}
keys = prih_summary['Описание 3'].unique()
for x in keys:
    idx = pd.IndexSlice
    filtr = df.loc[:, idx['Наполняемость',x]] == 'нет'
    shop_lsts[x] = df.xs(x, level=1, axis=1)[filtr].sort_values(by='Нормы').index.to_list()

# Остатки реальные + накладные
cols = ["КодСклада", "НазваниеСклада", "ТоварНо","СерийныйНомер","Проект"]
stock = pd.read_csv(r'C:\анаконда\tmp.csv', delimiter=';', encoding='utf-8',low_memory=False, usecols=cols)
stock = stock[stock.Проект.isin(['ЗОЛОТОЙ','ОРИОН','ЗОЛОТО','ТАЛАНТ', 'НОУНЕЙМ'])].sort_values('КодСклада')
del stock['Проект']

stock.dropna(inplace=True)
stock.КодСклада = stock.КодСклада.astype('int64')

stock_centr = stock[stock.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
stock = stock[~stock.НазваниеСклада.str.contains('центральный склад|опт',case = False)]
stock = stock[["КодСклада", "ТоварНо", "СерийныйНомер"]]
del stock_centr['КодСклада']

cols=["Серийный Но.","Склад-приемник","Номер ЗЛ"]
zab = pd.read_excel(r'C:\анаконда\zab.xlsx', skiprows=4, usecols=cols).sort_values(by="Номер ЗЛ",ascending = False)
zab.rename(columns={"Серийный Но.":'СерийныйНомер', "Склад-приемник":'КодСклада',"Номер ЗЛ":'ЗЛ'}, inplace=True) 
del zab['ЗЛ']

stock_centr = pd.merge(stock_centr,zab,on=['СерийныйНомер'],how = 'inner')#inner join оставит только неперемещенный товар
stock_centr = stock_centr[["КодСклада",  "ТоварНо","СерийныйНомер"]]
stock = pd.concat([stock,stock_centr])
#оставим только маги-топы и товы из накладной
unique_lst = list(set([val for vals in shop_lsts.values() for val in vals]))
stock = stock[stock.КодСклада.isin(unique_lst)].drop(columns='СерийныйНомер')
stock = stock[stock.ТоварНо.isin(prih_summary.Товар.tolist())]
#формат колонок кодсклада и товы
stock = pd.pivot_table(stock,index='КодСклада', columns='ТоварНо', aggfunc=len, fill_value=0).reset_index()
stock = stock.rename_axis(None, axis=1).set_index('КодСклада')
if set(prih_summary.Товар.values) != set(stock.columns):
    cols = set(prih_summary.Товар.values) - set(stock.columns)
    stock = stock.assign(**dict(zip(list(cols), np.zeros(len(cols), dtype='int64'))))

# Распределение
def distribution(tg):
    """распред по списку магов ТГ"""
    tov_dict = dict(prih_summary[prih_summary['Описание 3'] == tg].iloc[:,[1, 2]].values)
    cols = list(tov_dict.keys())    
    shops = shop_lsts[tg]
    tn_stock = stock.loc[shops, cols]
    emp_lst = []
    for col in tn_stock.columns.tolist():
        shop_lst = tn_stock[col].loc[lambda x: x == 0].index.tolist()
        lenth = len(shop_lst)
        diff = tov_dict[col] - lenth
        
        if diff > 0:
            shop_lst.extend(list(np.full(diff, 7171)))
        else:
            shop_lst = shop_lst[:tov_dict[col]]
            
        emp_lst.extend(shop_lst)
    return emp_lst
    
distr_lst = []
for x in list(shop_lsts.keys()):
    distr_lst.extend(distribution(x))

prih['Период реализации'] = distr_lst

# save and exit
name = 'C:/рушники/' + str(np.random.randint(5000, 10000)) + '.xlsx'
prih.to_excel(name, sheet_name='Движение товара', index=False)

In [4]:
df

Нормы                                          \
       ИФ_КОЛЬЦО ИФ_КОЛЬЦО ОБРУЧАЛЬНОЕ ИФ_КОЛЬЦО ПЕЧАТКА   
Код                                                        
92    407.175513             44.357453         85.035723   
157   345.960606             37.688738         94.640289   
161   613.279304             65.086188        111.087536   
244   971.805408            103.135894        211.248808   
252   566.237824             60.093763        110.472189   
...          ...                   ...               ...   
5559  610.582510             61.058251        101.763752   
5561  678.612048             72.537427        121.065697   
5562  312.313324             32.697049         49.527591   
5563  531.033276             56.357570         98.011443   
5565  203.527503             30.529125         30.529125   

                                                            Наполняемость  \
     ИФ_ПОДВЕС ДЕКОРАТИВНЫЙ ИФ_ПОДВЕС КУЛЬТОВЫЙ   ИФ_СЕРЬГИ     ИФ_КОЛЬЦО   
Код                                                                         
92               292.289387          113.668051  473.133318      1.051144   
157              305.291255          203.527503  356.173131      1.081048   
161              320.106827          121.129214  556.422306      1.084335   
244              487.028596          193.390620  766.407864      1.054738   
252              333.574900          129.604778  493.244091      1.031369   
...                     ...                 ...         ...           ...   
5559             417.231382           91.587376  508.818758      1.090762   
5561             306.916372          128.824355  532.845147      0.882684   
5562             220.379749           92.689420  300.660237      0.995795   
5563             350.074896           78.659021  442.435880      1.022535   
5565             142.469252           81.411001  274.762129      1.203769   

                                                                     ...  \
     ИФ_КОЛЬЦО ОБРУЧАЛЬНОЕ ИФ_КОЛЬЦО ПЕЧАТКА ИФ_ПОДВЕС ДЕКОРАТИВНЫЙ  ...   
Код                                                                  ...   
92                1.149750          1.093658               1.046908  ...   
157               1.140924          1.067199               1.008873  ...   
161               1.121590          1.035220               1.040278  ...   
244               1.076250          0.998822               1.059486  ...   
252               1.114924          0.977622               1.013266  ...   
...                    ...               ...                    ...  ...   
5559              1.277469          1.031802               1.030603  ...   
5561              0.937447          0.685578               0.703775  ...   
5562              1.376271          1.029729               0.748708  ...   
5563              1.029143          0.999883               0.936942  ...   
5565              1.113691          1.637780               1.130068  ...   

                 Нормы                                                         \
     БК_КОЛЬЦО ПЕЧАТКА БК_ПОДВЕС ДЕКОРАТИВНЫЙ БК_ПОДВЕС КУЛЬТОВЫЙ   БК_СЕРЬГИ   
Код                                                                             
92           54.162644              73.791429          187.382875  142.134626   
157          60.044090             101.645028          261.547985  203.539288   
161          68.219800              95.656025          247.236758  160.274662   
244         104.080593             106.088197          263.887291  276.018499   
252          58.581822              77.497311          202.025924  153.568307   
...                ...                    ...                 ...         ...   
5559         50.884822             122.123573          223.893216  183.185359   
5561         61.603049              90.804845          239.169136  181.899786   
5562         27.752388              67.081205          162.833373  112.042965   
5563         52.131934             113.351426         

In [98]:
def scubes(chunck):
    lst = [int(chunck[x]) ** 3 for x in range(len(chunck))]
    return sum(lst) % 2 == 0

def revrot(string, sz):
    if sz <= 0 or str == "":
        return ""
    else:
        chnks = len(string) //  sz
        lst = [string[x * sz :(x + 1) * sz] for x in range(chnks)]
        lst = [x[::-1] if scubes(x) else x[1:] + x[0] for x in lst]
        return ''.join(lst)
            
        

In [60]:
''.join(revrot("123456987654", 6))


'234561876549'

In [56]:
[x[:-1] if scubes(x) else x[1:] + x[0] for x in new]

['234561', '876549']

In [50]:
def scubes(chunck):
    lst = [int(chunck[x]) ** 3 for x in range(len(chunck))]
    return sum(lst) % 2 == 0

In [90]:
scubes('5630')

True

In [99]:
revrot("563000655734469485", 4)

'0365065073456944'

In [93]:
string, sz = "563000655734469485", 4
chnks = len(string) //  sz
lst = [string[x * sz :(x + 1) * sz] for x in range(chnks)]
#lst = [x[:-1] if scubes(x) else x[1:] + x[0] for x in lst]

In [97]:
lst[0][::-1]

'0365'